In [ ]:
import numpy as np
from pathlib import Path

from change_of_basis import cmat_of_tmat
from change_of_basis import tmat_of_cmat
from materials import get_materials_Cvec
from safe_module import closest
from utilities import print_matrix_matlab
from utilities import sm2v
from utilities import v2sm

In [ ]:
write = False

In [ ]:
map = "Brown"
database_type = "SymGroups" # options are [SymGroups, t20, t10, t2_5]

In [ ]:
def footer(filename, textlines, c_vec):
    
    with open(filename, 'a') as f:
        
        for textline in textlines:
            f.write(f'\n{textline}')
            
        f.write('  if (iflag_aniso == IANISOTROPY_MODEL2) then\n')
        c = 0
        for i in range(1,7):
            for j in range(i,7):
                f.write(f'    c{i}{j} = {c_vec[c]:17.12f} * 1d9\n')
                c += 1
        f.write('  endif\n')
        f.write('\n')
        f.write('  end subroutine model_aniso')

In [ ]:
root_dir = Path.cwd().resolve().parent.parent
base_dir = root_dir / "Scripts" / "Data" / "SPECFEM3D_CARTESIAN" / "model_aniso"
source_file = base_dir / "Template" / "model_aniso.f90"
destination_dir = base_dir / f"{map}_{database_type}"

if write:
    ! mkdir -p {destination_dir}

In [ ]:
Cvec, Cvec_stdv, *_ = get_materials_Cvec(material=map)

In [ ]:
Cmat = v2sm(Cvec)
Tmat = tmat_of_cmat(Cmat)

In [ ]:
# SymGroup models

if database_type == "SymGroups":

    node_sequence    = ['TRIV','MONO','ORTH','TET','XISO','ISO']
    
    Tmat_nodes = []
    for sigma in node_sequence:
            print(f'adding node {sigma} ....')
            if sigma == "TRIV":
                Tmat_nodes.append(Tmat)
            else:
                Tmat_nodes.append(closest(Tmat, sigma))

In [ ]:
# SymGroups models

if database_type == "SymGroups":

    symmetry_classes = ['trivial', 'monoclinic', 'orthorhombic', 'tetragonal',
                        'transversely isotropic', 'isotropic']
    models = ['TRIV', 'MONO_TRIV', 'MONO', 'ORTH_MONO', 'ORTH', 'TET_ORTH', 'TET',
              'XISO_TET', 'XISO', 'ISO_XISO', 'ISO']
    
    for i, model in enumerate(models):
    
        print(f'computing model {model} ....')
    
        if i%2==0:
            j = round(i/2)
            t_mat_test = Tmat_nodes[j]
        else:
            j = round((i-1)/2)
            t_mat1 = Tmat_nodes[j]
            t_mat2 = Tmat_nodes[j+1]
            t_mat_test = 0.5 * t_mat1 + 0.5 * t_mat2
    
        c_mat_test = cmat_of_tmat(t_mat_test)
        c_vec_test = sm2v(c_mat_test)
    
        print(f'printing model {model} ....')
        print_matrix_matlab(c_mat_test)
    
        if write:
            print(f'writing model {model} ....')
            filename = f'model_aniso_{model}.f90'
            destination_file = destination_dir / filename
            ! cp {source_file} {destination_file}
        
            if i%2==0:
                if j==0:
                    textlines = ['! overwrite perturbation model 2 with the '
                                 f'{map}2016 map\n']
                else:
                    textlines = [f'! overwrite perturbation model 2 with closest'
                                 f' {symmetry_classes[j]} map to the {map}2016'
                                 f' map\n']
            else:
                textlines = [f'! overwrite perturbation model 2 with map halfway'
                             f' between closest {symmetry_classes[j+1]} and'
                             f' {symmetry_classes[j]} map',
                             f'! to the {map}2016 map\n']
                
            footer(destination_file, textlines, c_vec_test)
            print("done.\n")

In [ ]:
# t** models

if database_type[0] == "t":
    if database_type == "t20":
        models = ['160', '140', '120', '100', '80', '60', '40', '20', '00']
        t_min  = -3/5
        t_max  = 1
        dt     = 1/5
    elif database_type == "t10":
        models = ['160', '150', '140', '130', '120', '110', '100',
                   '90', '80', '70', '60', '50', '40', '30', '20', '10', '00']
        t_min  = -3/5
        t_max  = 1
        dt     = 1/10
    elif database_type == "t2_5":
        models = ['20', '17_5', '15', '12_5', '10', '7_5', '5', '2_5', '00']
        t_min  = 80 / 100
        t_max  = 100 / 100
        dt     = 2.5 / 100
    else:
        raise ValueError('database_type must be one of [t20, t10, t2_5]')

    t_mat1 = Tmat
    t_mat2 = closest(Tmat, 'ISO')
    
    npts     = round( (t_max - t_min) / dt + 1 )
    frac     = np.linspace(t_min,t_max,npts)
    
    for i, model in enumerate(models):
    
        print(f'computing model t{model} ....')
    
        t_mat_test = (1-frac[i]) * t_mat1 + (frac[i]) * t_mat2
        c_mat_test = cmat_of_tmat(t_mat_test)
        c_vec_test = sm2v(c_mat_test)
    
        print(f'printing model t{model} ....')
        print_matrix_matlab(c_mat_test)
    
        if write:
            print(f'writing model t{model} ....')
            filename = f'model_aniso_t{model}.f90'
            destination_file = destination_dir / filename
            ! cp {source_file} {destination_file}
            
            if model == '00':
                textlines = ['! overwrite perturbation model 2 with closest isotropic'
                             f' map to the {map}2016 map\n']
            elif model == '100':
                textlines = ['! overwrite perturbation model 2 with the '
                             f'{map}2016 map\n']
            else:
                textlines = [f'! overwrite perturbation model 2 with map with {model}%'
                             f' anisotropy of {map}2016 map\n']
    
            footer(destination_file, textlines, c_vec_test)
            print("done.\n")